# Problem solving by searching
Demonstration of Dijkstra's, A* Search and Tabu Search algorithms for finding the shortest path between given stops A and B.

## Task 1 - Dijkstra's and A* Search algorithm
The application should accept input in form of 4 variables:
- the starting stop A (marked as `start_stop`)
- the ending stop B (marked as `goal`)
- optimization criterion (value `t` denotes travel time minimization, value `p` means minimization of the number of transfers)
- starting time.

### Classes and helper-functions implementation
Before we implement the algorithms, let's first prepare environment to work. We will use the `Connection` class to store each row of `connection_graph.csv` file, and the `Graph` class to load and store the connections. We will also use `PriorityQueue` for stops (and lines) prioritization in Dijkstra's and A* Search algorithms.

In [1]:
import csv
import heapq
import argparse
from datetime import datetime, timedelta
import math
import time
import sys
import random

In [2]:
def print_path(path):
    for connection in path:
        print(f"{connection.start_stop} -> {connection.end_stop}, Line: {connection.line}, Departure: {connection.departure_time.time()}, Arrival: {connection.arrival_time.time()}")

def time_diff_seconds(time1, time2):
    return (time1 - time2).total_seconds()

def calculate_distance(lat1, lon1, lat2, lon2):
    return math.sqrt((lat1 - lat2) ** 2 + (lon1 - lon2) ** 2)

def calculate_distance_stops(stop1, stop2, graph):
    lat1, lon1 = graph.get_connections(stop1)[0].start_stop_lat, graph.get_connections(stop1)[0].start_stop_lon
    lat2, lon2 = graph.get_connections(stop2)[0].start_stop_lat, graph.get_connections(stop2)[0].start_stop_lon
    return calculate_distance(lat1, lon1, lat2, lon2)

def choose_common_line(graph, current, goal):
    current_lines = {connection.line for connection in graph.get_connections(current)}
    end_lines = {connection.line for connection in graph.get_connections(goal)}
    common_lines = current_lines.intersection(end_lines)
    if common_lines:
        return list(common_lines)[0]
    else:
        return None
    
def parse_time(time_str):
    try:
        hours, minutes, seconds = map(int, time_str.split(':'))
    except ValueError:
        try:
            hours, minutes = map(int, time_str.split(':'))
            seconds = 0
        except ValueError:
            raise argparse.ArgumentTypeError("Invalid time format. Please use HH:MM or HH:MM:SS.")
    base_date = datetime(2023, 3, 2)
    if hours >= 24:
        hours -= 24
        return base_date.replace(hour=hours, minute=minutes, second=seconds) + timedelta(days=1)
    elif hours < 0:
        hours += 24
        return base_date.replace(hour=hours, minute=minutes, second=seconds) - timedelta(days=1)
    else:
        return base_date.replace(hour=hours, minute=minutes, second=seconds)

In [3]:
class Connection:
    def __init__(self, company, line, departure_time, arrival_time, start_stop, end_stop, start_stop_lat, start_stop_lon, end_stop_lat, end_stop_lon):
        self.company = company
        self.line = line
        self.departure_time = departure_time
        self.arrival_time = arrival_time
        self.start_stop = start_stop
        self.end_stop = end_stop
        self.start_stop_lat = start_stop_lat
        self.start_stop_lon = start_stop_lon
        self.end_stop_lat = end_stop_lat
        self.end_stop_lon = end_stop_lon
    
    def __str__(self) -> str:
        return f"Company: {self.company}, Line: {self.line}, Departure: {self.departure_time}, Arrival: {self.arrival_time}, Start Stop: {self.start_stop}, End Stop: {self.end_stop}, Start Stop Lat: {self.start_stop_lat}, Start Stop Lon: {self.start_stop_lon}, End Stop Lat: {self.end_stop_lat}, End Stop Lon: {self.end_stop_lon}"


In [4]:
class Graph:
    def __init__(self):
        self.graph = {}

    def load_connections(self, file_path):
        # Loading connections from CSV into self.connections
        with open(file_path, newline='', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                departure_time = parse_time(row['departure_time'])
                arrival_time = parse_time(row['arrival_time'])
                connection = Connection(
                    row['company'],
                    row['line'],
                    departure_time,
                    arrival_time,
                    row['start_stop'],
                    row['end_stop'],
                    float(row['start_stop_lat']),
                    float(row['start_stop_lon']),
                    float(row['end_stop_lat']),
                    float(row['end_stop_lon'])
                )
                if connection.start_stop not in self.graph:
                    self.graph[connection.start_stop] = []
                self.graph[connection.start_stop].append(connection)

    def get_connections(self, stop):
        return self.graph.get(stop, [])

In [5]:
class PriorityQueue:
    def __init__(self):
        self.elements = []

    def empty(self):
        return not self.elements

    def put(self, item, priority):
        heapq.heappush(self.elements, (priority, item))

    def get(self):
        return heapq.heappop(self.elements)[1]

In [5]:
# Loading connections from CSV file
graph = Graph()
graph.load_connections('connection_graph.csv')

### Dijkstra's Algorithm
Shortest path search from A to B using Dijkstra's algorithm, based on travel time criterion.

In [9]:
def dijkstra(graph, start, goal, start_time):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from = {}
    cost_so_far = {start: 0}
    path = []
    
    journey_start_time = start_time
    
    while not frontier.empty():
        current = frontier.get()
        
        # modification: early exit
        if current == goal:
            break
        
        for connection in graph.get_connections(current):
            next_stop = connection.end_stop
            departure_time = connection.departure_time
            arrival_time = connection.arrival_time
            
            if departure_time >= start_time:
                waiting_time = time_diff_seconds(departure_time, start_time)
                travel_time = time_diff_seconds(arrival_time, departure_time)
                new_cost = cost_so_far[current] + travel_time + waiting_time
                if next_stop not in cost_so_far or new_cost < cost_so_far[next_stop]:
                    cost_so_far[next_stop] = new_cost
                    priority = new_cost
                    frontier.put(next_stop, priority)
                    came_from[next_stop] = (current, connection)
            if current in came_from:
                start_time = (came_from[current][1]).arrival_time 
                            
    current = goal
    if current not in came_from:
        return []
    while current != start:
        path.append(came_from[current][1])
        current = came_from[current][0]
    
    path.reverse()
    
    journey_end_time = path[-1].arrival_time
    total_journey_time = journey_end_time - journey_start_time
    
    return path, total_journey_time

Now let's take some sample data to see how it works.

In [10]:
start_stop = "Kasprowicza"
goal = "Kominiarska"
criterion = "t"
if criterion != 'p' and criterion != 't':
    raise argparse.ArgumentTypeError("Optimization criterion should be either 't' or 'p'.")

start_time = "12:07:00"
start_time = parse_time(start_time)

minimized_criterion_value = "Travel Time" if criterion == 't' else "Number of Transfers"

start = time.time()
dijkstra_path, dijkstra_journey = dijkstra(graph, start_stop, goal, start_time)
end = time.time()
req_time = end - start


print_path(dijkstra_path)

print()
print(f"Starting time: {start_time}")
print(f"Total journey time: {dijkstra_journey}")
print(f"Minimized Criterion: {minimized_criterion_value}", file=sys.stderr)
print(f"Time Required: {req_time}", file=sys.stderr)

Kasprowicza -> Syrokomli, Line: 105, Departure: 12:07:00, Arrival: 12:08:00
Syrokomli -> Pola, Line: 105, Departure: 12:08:00, Arrival: 12:09:00
Pola -> Broniewskiego, Line: 105, Departure: 12:09:00, Arrival: 12:11:00
Broniewskiego -> Bałtycka, Line: 105, Departure: 12:11:00, Arrival: 12:13:00
Bałtycka -> Bezpieczna, Line: 105, Departure: 12:13:00, Arrival: 12:15:00
Bezpieczna -> Paprotna, Line: 105, Departure: 12:15:00, Arrival: 12:17:00
Paprotna -> Irysowa, Line: 105, Departure: 12:17:00, Arrival: 12:18:00
Irysowa -> Obornicka (Obwodnica), Line: 105, Departure: 12:18:00, Arrival: 12:20:00
Obornicka (Obwodnica) -> Ostowa (Muzeum Militarne), Line: 105, Departure: 12:20:00, Arrival: 12:21:00
Ostowa (Muzeum Militarne) -> Pełczyńska (Stacja kolejowa), Line: 105, Departure: 12:21:00, Arrival: 12:22:00
Pełczyńska (Stacja kolejowa) -> Kominiarska, Line: 105, Departure: 12:22:00, Arrival: 12:23:00

Starting time: 2023-03-02 12:07:00
Total journey time: 0:16:00


Minimized Criterion: Travel Time
Time Required: 0.2820556163787842


In [11]:
start_stop = "GALERIA DOMINIKAŃSKA"
goal = "Bezpieczna"
criterion = "t"
if criterion != 'p' and criterion != 't':
    raise argparse.ArgumentTypeError("Optimization criterion should be either 't' or 'p'.")

start_time = "24:09:11"
start_time = parse_time(start_time)

minimized_criterion_value = "Travel Time" if criterion == 't' else "Number of Transfers"

start = time.time()
dijkstra_path, dijkstra_journey = dijkstra(graph, start_stop, goal, start_time)
end = time.time()
req_time = end - start


print_path(dijkstra_path)

print()
print(f"Starting time: {start_time}")
print(f"Total journey time: {dijkstra_journey}")
print(f"Minimized Criterion: {minimized_criterion_value}", file=sys.stderr)
print(f"Time Required: {req_time}", file=sys.stderr)

GALERIA DOMINIKAŃSKA -> Świdnicka, Line: 110, Departure: 00:15:00, Arrival: 00:16:00
Świdnicka -> Rynek, Line: 110, Departure: 00:16:00, Arrival: 00:18:00
Rynek -> Mosty Pomorskie, Line: 110, Departure: 00:18:00, Arrival: 00:19:00
Mosty Pomorskie -> Pomorska, Line: 110, Departure: 00:19:00, Arrival: 00:20:00
Pomorska -> pl. Strzelecki, Line: 110, Departure: 00:21:00, Arrival: 00:23:00
pl. Strzelecki -> Kleczkowska, Line: 110, Departure: 00:23:00, Arrival: 00:25:00
Kleczkowska -> Bałtycka, Line: 110, Departure: 00:25:00, Arrival: 00:28:00
Bałtycka -> Bezpieczna, Line: 110, Departure: 00:28:00, Arrival: 00:29:00

Starting time: 2023-03-03 00:09:11
Total journey time: 0:19:49


Minimized Criterion: Travel Time
Time Required: 0.11561179161071777


In [13]:
start_stop = "Pereca"
goal = "KROMERA"
criterion = "t"
if criterion != 'p' and criterion != 't':
    raise argparse.ArgumentTypeError("Optimization criterion should be either 't' or 'p'.")

start_time = "9:07"
start_time = parse_time(start_time)

minimized_criterion_value = "Travel Time" if criterion == 't' else "Number of Transfers"

start = time.time()
dijkstra_path, dijkstra_journey = dijkstra(graph, start_stop, goal, start_time)
end = time.time()
req_time = end - start


print_path(dijkstra_path)

print()
print(f"Starting time: {start_time}")
print(f"Total journey time: {dijkstra_journey}")
print(f"Minimized Criterion: {minimized_criterion_value}", file=sys.stderr)
print(f"Time Required: {req_time}", file=sys.stderr)

Pereca -> Grabiszyńska, Line: 11, Departure: 09:09:00, Arrival: 09:10:00
Grabiszyńska -> Kolejowa, Line: 11, Departure: 09:10:00, Arrival: 09:11:00
Kolejowa -> pl. Legionów, Line: 11, Departure: 09:11:00, Arrival: 09:13:00
pl. Legionów -> Narodowe Forum Muzyki, Line: 11, Departure: 09:13:00, Arrival: 09:15:00
Narodowe Forum Muzyki -> Zamkowa, Line: 11, Departure: 09:15:00, Arrival: 09:16:00
Zamkowa -> Świdnicka, Line: 11, Departure: 09:16:00, Arrival: 09:18:00
Świdnicka -> GALERIA DOMINIKAŃSKA, Line: 11, Departure: 09:18:00, Arrival: 09:21:00
GALERIA DOMINIKAŃSKA -> pl. Nowy Targ, Line: 8, Departure: 09:21:00, Arrival: 09:22:00
pl. Nowy Targ -> Hala Targowa, Line: 8, Departure: 09:22:00, Arrival: 09:23:00
Hala Targowa -> pl. Bema, Line: 8, Departure: 09:23:00, Arrival: 09:26:00
pl. Bema -> Na Szańcach, Line: 8, Departure: 09:26:00, Arrival: 09:27:00
Na Szańcach -> Jedności Narodowej, Line: 8, Departure: 09:27:00, Arrival: 09:28:00
Jedności Narodowej -> Nowowiejska, Line: 11, Departure:

Minimized Criterion: Travel Time
Time Required: 1.4258911609649658


In [14]:
start_stop = "Gąsiorowskiego"
goal = "Komuny Paryskiej"
criterion = "t"
if criterion != 'p' and criterion != 't':
    raise argparse.ArgumentTypeError("Optimization criterion should be either 't' or 'p'.")

start_time = "14:55"
start_time = parse_time(start_time)

minimized_criterion_value = "Travel Time" if criterion == 't' else "Number of Transfers"

start = time.time()
dijkstra_path, dijkstra_journey = dijkstra(graph, start_stop, goal, start_time)
end = time.time()
req_time = end - start


print_path(dijkstra_path)

print()
print(f"Starting time: {start_time}")
print(f"Total journey time: {dijkstra_journey}")
print(f"Minimized Criterion: {minimized_criterion_value}", file=sys.stderr)
print(f"Time Required: {req_time}", file=sys.stderr)

Gąsiorowskiego -> Jutrosińska, Line: 144, Departure: 14:58:00, Arrival: 14:59:00
Jutrosińska -> Mochnackiego, Line: K, Departure: 14:59:00, Arrival: 15:00:00
Mochnackiego -> Kamieńskiego, Line: K, Departure: 15:00:00, Arrival: 15:02:00
Kamieńskiego -> Broniewskiego, Line: K, Departure: 15:02:00, Arrival: 15:04:00
Broniewskiego -> Trzebnicka, Line: 1, Departure: 15:04:00, Arrival: 15:07:00
Trzebnicka -> DWORZEC NADODRZE, Line: 1, Departure: 15:07:00, Arrival: 15:09:00
DWORZEC NADODRZE -> Słowiańska, Line: 1, Departure: 15:10:00, Arrival: 15:12:00
Słowiańska -> Nowowiejska, Line: 1, Departure: 15:12:00, Arrival: 15:14:00
Nowowiejska -> Wyszyńskiego, Line: 1, Departure: 15:14:00, Arrival: 15:16:00
Wyszyńskiego -> Prusa, Line: 1, Departure: 15:16:00, Arrival: 15:17:00
Prusa -> Piastowska, Line: 1, Departure: 15:17:00, Arrival: 15:19:00
Piastowska -> PL. GRUNWALDZKI, Line: 1, Departure: 15:19:00, Arrival: 15:22:00
PL. GRUNWALDZKI -> most Grunwaldzki, Line: 13, Departure: 15:23:00, Arrival: 

Minimized Criterion: Travel Time
Time Required: 0.6757371425628662


Here we can test it with our own input:

In [16]:
start_stop = input("Please provide the starting stop: ")
goal = input("Please provide the ending stop: ")
#criterion = input(f"Optimization criterion:\n\tt (travel time)\n\tp (number of transfers)")
#if criterion != 'p' and criterion != 't':
#    raise argparse.ArgumentTypeError("Optimization criterion should be either 't' or 'p'.")

start_time = input("Please provide time in form HH:MM or HH:MM:SS: ")
start_time = parse_time(start_time)


minimized_criterion_value = "Travel Time" if criterion == 't' else "Number of Transfers"

In [22]:
dijkstra_path, dijkstra_journey = dijkstra(graph, start_stop, goal, start_time)

In [23]:
print_path(dijkstra_path)

print()
print(f"Starting time: {start_time}")
print(f"Total journey time: {dijkstra_journey}")

Kasprowicza -> Syrokomli, Line: 105, Departure: 12:07:00, Arrival: 12:08:00
Syrokomli -> Pola, Line: 105, Departure: 12:08:00, Arrival: 12:09:00
Pola -> Broniewskiego, Line: 105, Departure: 12:09:00, Arrival: 12:11:00
Broniewskiego -> Bałtycka, Line: 105, Departure: 12:11:00, Arrival: 12:13:00
Bałtycka -> Bezpieczna, Line: 105, Departure: 12:13:00, Arrival: 12:15:00
Bezpieczna -> Paprotna, Line: 105, Departure: 12:15:00, Arrival: 12:17:00
Paprotna -> Irysowa, Line: 105, Departure: 12:17:00, Arrival: 12:18:00
Irysowa -> Obornicka (Obwodnica), Line: 105, Departure: 12:18:00, Arrival: 12:20:00
Obornicka (Obwodnica) -> Ostowa (Muzeum Militarne), Line: 105, Departure: 12:20:00, Arrival: 12:21:00
Ostowa (Muzeum Militarne) -> Pełczyńska (Stacja kolejowa), Line: 105, Departure: 12:21:00, Arrival: 12:22:00
Pełczyńska (Stacja kolejowa) -> Kominiarska, Line: 105, Departure: 12:22:00, Arrival: 12:23:00
Starting time: 2023-03-02 12:07:00
Total journey time: 0:16:00


### A* Search Algorithm
So basically, A* search algorithm is just Dijkstra's with heuristic...
Here we are going to discuss both criteria: time optimization and number of changes optimization.

#### A* Search with Time Optimization Criterion
We are going to use heuristic that calculates estimated time needed to arrive at the goal based on velociy gained in the last transfer and distance between current stop and the goal. First, we calculate the distance between current stop, and the next one, we also calculate time needed for this connection, and from that we can calculate the velocity. Then we calculate the distance between the current stop and the goal, and with given velocity, we can estimate the arrival time. 

In [15]:
def heuristic_t(graph, current_connection, goal):
        current_lat = current_connection.start_stop_lat
        current_lon = current_connection.start_stop_lon
        next_lat = current_connection.end_stop_lat
        next_lon = current_connection.end_stop_lon
        
        last_distance = calculate_distance(current_lat, current_lon, next_lat, next_lon)
        
        arrival = current_connection.arrival_time
        departure = current_connection.departure_time
        
        last_time = time_diff_seconds(arrival, departure)
        last_velocity = last_distance / last_time if last_time > 0 else float('inf')
        
        goal_connection = graph.get_connections(goal)[0]
        goal_lat = goal_connection.start_stop_lat
        goal_lon = goal_connection.start_stop_lon
        
        to_goal_distance = calculate_distance(goal_lat, goal_lon, current_lat, current_lon)
        if last_velocity == 0:
            estimated_time = float('inf')
        elif last_velocity == float('inf'):
            estimated_time = 0 
        else:
            estimated_time = to_goal_distance / last_velocity
        return estimated_time

In [16]:
def a_star_time(graph, start, goal, start_time):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from = {}
    cost_so_far = {start: 0}
    path = []

    journey_start_time = start_time
        
    while not frontier.empty():
        current = frontier.get()
        
        # modification: early exit
        if current == goal:
            break
        
        for connection in graph.get_connections(current):
            next_stop = connection.end_stop
            departure_time = connection.departure_time
            arrival_time = connection.arrival_time
            
            if departure_time >= start_time:
                waiting_time = time_diff_seconds(departure_time, start_time)
                travel_time = time_diff_seconds(arrival_time, departure_time)
                new_cost = cost_so_far[current] + travel_time + waiting_time
                if next_stop not in cost_so_far or new_cost < cost_so_far[next_stop]:
                    cost_so_far[next_stop] = new_cost
                    priority = new_cost + heuristic_t(graph, connection, goal) # here we just add heuristic
                    frontier.put(next_stop, priority)
                    came_from[next_stop] = (current, connection)
            if current in came_from:
                start_time = (came_from[current][1]).arrival_time 
                            
    current = goal
    if current not in came_from:
        return []
    while current != start:
        path.append(came_from[current][1])
        current = came_from[current][0]
    
    path.reverse()
    
    journey_end_time = path[-1].arrival_time
    total_journey_time = journey_end_time - journey_start_time
    
    return path, total_journey_time  

And now, let's see some sample data to see how it works.

In [17]:
start_stop = "GALERIA DOMINIKAŃSKA"
goal = "Bezpieczna"
criterion = "t"
if criterion != 'p' and criterion != 't':
    raise argparse.ArgumentTypeError("Optimization criterion should be either 't' or 'p'.")

start_time = "24:09:11"
start_time = parse_time(start_time)

minimized_criterion_value = "Travel Time" if criterion == 't' else "Number of Transfers"

start = time.time()
a_star_t_path, a_star_t_journey = a_star_time(graph, start_stop, goal, start_time)
end = time.time()
req_time = end - start


print_path(a_star_t_path)

print()
print(f"Starting time: {start_time}")
print(f"Total journey time: {a_star_t_journey}")
print(f"Minimized Criterion: {minimized_criterion_value}", file=sys.stderr)
print(f"Time Required: {req_time}", file=sys.stderr)

GALERIA DOMINIKAŃSKA -> Świdnicka, Line: 110, Departure: 00:15:00, Arrival: 00:16:00
Świdnicka -> Rynek, Line: 110, Departure: 00:16:00, Arrival: 00:18:00
Rynek -> Mosty Pomorskie, Line: 110, Departure: 00:18:00, Arrival: 00:19:00
Mosty Pomorskie -> Pomorska, Line: 110, Departure: 00:19:00, Arrival: 00:20:00
Pomorska -> pl. Strzelecki, Line: 110, Departure: 00:21:00, Arrival: 00:23:00
pl. Strzelecki -> Kleczkowska, Line: 110, Departure: 00:23:00, Arrival: 00:25:00
Kleczkowska -> Bałtycka, Line: 110, Departure: 00:25:00, Arrival: 00:28:00
Bałtycka -> Bezpieczna, Line: 110, Departure: 00:28:00, Arrival: 00:29:00

Starting time: 2023-03-03 00:09:11
Total journey time: 0:19:49


Minimized Criterion: Travel Time
Time Required: 0.05382084846496582


In [23]:
start_stop = "Dubois"
goal = "LEŚNICA"
criterion = "t"
if criterion != 'p' and criterion != 't':
    raise argparse.ArgumentTypeError("Optimization criterion should be either 't' or 'p'.")

start_time = "14:00"
start_time = parse_time(start_time)

minimized_criterion_value = "Travel Time" if criterion == 't' else "Number of Transfers"

start = time.time()
a_star_t_path, a_star_t_journey = a_star_time(graph, start_stop, goal, start_time)
end = time.time()
req_time = end - start


print_path(a_star_t_path)

print()
print(f"Starting time: {start_time}")
print(f"Total journey time: {a_star_t_journey}")
print(f"Minimized Criterion: {minimized_criterion_value}", file=sys.stderr)
print(f"Time Required: {req_time}", file=sys.stderr)

Dubois -> Rynek, Line: 132, Departure: 14:05:00, Arrival: 14:09:00
Rynek -> PL. JANA PAWŁA II, Line: 132, Departure: 14:09:00, Arrival: 14:12:00
PL. JANA PAWŁA II -> Młodych Techników, Line: 22, Departure: 14:12:00, Arrival: 14:14:00
Młodych Techników -> pl. Strzegomski (Muzeum Współczesne), Line: 22, Departure: 14:14:00, Arrival: 14:15:00
pl. Strzegomski (Muzeum Współczesne) -> Wrocław Mikołajów (Zachodnia), Line: 22, Departure: 14:15:00, Arrival: 14:17:00
Wrocław Mikołajów (Zachodnia) -> Niedźwiedzia, Line: 22, Departure: 14:17:00, Arrival: 14:19:00
Niedźwiedzia -> Małopanewska, Line: 22, Departure: 14:19:00, Arrival: 14:20:00
Małopanewska -> Kwiska, Line: 22, Departure: 14:20:00, Arrival: 14:21:00
Kwiska -> DH Astra, Line: 22, Departure: 14:21:00, Arrival: 14:23:00
DH Astra -> Park Zachodni, Line: 22, Departure: 14:23:00, Arrival: 14:24:00
Park Zachodni -> Bajana, Line: 22, Departure: 14:24:00, Arrival: 14:25:00
Bajana -> Metalowców, Line: 22, Departure: 14:25:00, Arrival: 14:27:00


Minimized Criterion: Travel Time
Time Required: 0.878741979598999


In [27]:
start_stop = "KSIĘŻE MAŁE"
goal = "Rzemieślnicza"
criterion = "t"
if criterion != 'p' and criterion != 't':
    raise argparse.ArgumentTypeError("Optimization criterion should be either 't' or 'p'.")

start_time = "18:05"
start_time = parse_time(start_time)

minimized_criterion_value = "Travel Time" if criterion == 't' else "Number of Transfers"

start = time.time()
a_star_t_path, a_star_t_journey = a_star_time(graph, start_stop, goal, start_time)
end = time.time()
req_time = end - start


print_path(a_star_t_path)

print()
print(f"Starting time: {start_time}")
print(f"Total journey time: {a_star_t_journey}")
print(f"Minimized Criterion: {minimized_criterion_value}", file=sys.stderr)
print(f"Time Required: {req_time}", file=sys.stderr)

KSIĘŻE MAŁE -> Karwińska, Line: 124, Departure: 18:06:00, Arrival: 18:08:00
Karwińska -> Park Wschodni, Line: 3, Departure: 18:08:00, Arrival: 18:09:00
Park Wschodni -> Armii Krajowej, Line: 3, Departure: 18:09:00, Arrival: 18:10:00
Armii Krajowej -> KRAKOWSKA (Centrum handlowe), Line: 114, Departure: 18:10:00, Arrival: 18:11:00
KRAKOWSKA (Centrum handlowe) -> Krakowska, Line: 114, Departure: 18:11:00, Arrival: 18:12:00
Krakowska -> Na Niskich Łąkach, Line: 114, Departure: 18:12:00, Arrival: 18:13:00
Na Niskich Łąkach -> Świstackiego, Line: 114, Departure: 18:13:00, Arrival: 18:14:00
Świstackiego -> Komuny Paryskiej, Line: 114, Departure: 18:14:00, Arrival: 18:15:00
Komuny Paryskiej -> Komuny Paryskiej (szkoła), Line: 114, Departure: 18:15:00, Arrival: 18:16:00
Komuny Paryskiej (szkoła) -> Wzgórze Partyzantów, Line: 114, Departure: 18:16:00, Arrival: 18:17:00
Wzgórze Partyzantów -> Renoma, Line: 106, Departure: 18:17:00, Arrival: 18:19:00
Renoma -> pl. Orląt Lwowskich, Line: 106, Depar

Minimized Criterion: Travel Time
Time Required: 0.4867253303527832


In [29]:
start_stop = "Psie Pole"
goal = "Karwińska"
criterion = "t"
if criterion != 'p' and criterion != 't':
    raise argparse.ArgumentTypeError("Optimization criterion should be either 't' or 'p'.")

start_time = "9:11:01"
start_time = parse_time(start_time)

minimized_criterion_value = "Travel Time" if criterion == 't' else "Number of Transfers"

start = time.time()
a_star_t_path, a_star_t_journey = a_star_time(graph, start_stop, goal, start_time)
end = time.time()
req_time = end - start


print_path(a_star_t_path)

print()
print(f"Starting time: {start_time}")
print(f"Total journey time: {a_star_t_journey}")
print(f"Minimized Criterion: {minimized_criterion_value}", file=sys.stderr)
print(f"Time Required: {req_time}", file=sys.stderr)

Psie Pole -> Zielna, Line: 911, Departure: 09:12:00, Arrival: 09:13:00
Zielna -> C.H. Korona, Line: 911, Departure: 09:13:00, Arrival: 09:15:00
C.H. Korona -> Brücknera, Line: 121, Departure: 09:15:00, Arrival: 09:17:00
Brücknera -> Kwidzyńska, Line: 121, Departure: 09:17:00, Arrival: 09:19:00
Kwidzyńska -> Zacisze, Line: 121, Departure: 09:19:00, Arrival: 09:20:00
Zacisze -> Śniadeckich, Line: 121, Departure: 09:20:00, Arrival: 09:21:00
Śniadeckich -> Kochanowskiego, Line: 121, Departure: 09:21:00, Arrival: 09:23:00
Kochanowskiego -> PL. GRUNWALDZKI, Line: 911, Departure: 09:24:00, Arrival: 09:26:00
PL. GRUNWALDZKI -> Kliniki - Politechnika Wrocławska, Line: 2, Departure: 09:26:00, Arrival: 09:28:00
Kliniki - Politechnika Wrocławska -> Hala Stulecia, Line: 2, Departure: 09:28:00, Arrival: 09:30:00
Hala Stulecia -> ZOO, Line: 2, Departure: 09:30:00, Arrival: 09:31:00
ZOO -> Tramwajowa, Line: 2, Departure: 09:31:00, Arrival: 09:32:00
Tramwajowa -> Chełmońskiego, Line: 2, Departure: 09:3

Minimized Criterion: Travel Time
Time Required: 0.4818589687347412


Here we can test it with our own input:

In [ ]:
start_stop = input("Please provide the starting stop: ")
goal = input("Please provide the ending stop: ")
#criterion = input(f"Optimization criterion:\n\tt (travel time)\n\tp (number of transfers)")
#if criterion != 'p' and criterion != 't':
#    raise argparse.ArgumentTypeError("Optimization criterion should be either 't' or 'p'.")

start_time = input("Please provide time in form HH:MM or HH:MM:SS: ")
start_time = parse_time(start_time)


minimized_criterion_value = "Travel Time" if criterion == 't' else "Number of Transfers"

In [ ]:
a_star_t_path, a_star_t_journey = a_star_time(graph, start_stop, goal, start_time)

In [ ]:
print_path(a_star_t_path)

print()
print(f"Starting time: {start_time}")
print(f"Total journey time: {dijkstra_journey}")

#### A* Search with Number of Transfers Optimization Criterion
Here we are going to use heuristic that checks if there are common lines at the destination stop and the current one. If so, we return 0, as we may not need to change lines, and otherwise we choose the minimum number of transfers needed among number of lines leaving the current node and the end node.

In [30]:
def heuristic_p(graph, current_connection, goal):
    current_lines = {connection.line for connection in graph.get_connections(current_connection.start_stop)}
    end_lines = {connection.line for connection in graph.get_connections(goal)}
        
    common_lines = current_lines.intersection(end_lines)
    if common_lines:
        # if there are common lines, return 0 transfers
        return 0
    else:
        # calculate the minimum number of transfers needed based on the number of lines at each stop
        return min(len(current_lines), len(end_lines))

In [43]:
def a_star_transfer(graph, start, goal, start_time):
    first_line = choose_common_line(graph, start, goal)
    frontier = PriorityQueue()
    frontier.put((start, first_line), 0)
    came_from = {}
    cost_so_far = {start: 0}
    path = []
    
    journey_start_time = start_time
    
    while not frontier.empty():
        current_stop, current_line = frontier.get()
        
        # modification: early exit
        if current_stop == goal:
            break
        
        for connection in graph.get_connections(current_stop):
            next_stop = connection.end_stop

            departure_time = connection.departure_time
            line = connection.line
            
            if departure_time >= start_time:
                transfer_count = 1 if (current_line != line) else 0
                new_cost = cost_so_far[current_stop] + transfer_count
                if next_stop not in cost_so_far or new_cost < cost_so_far[next_stop]:
                    cost_so_far[next_stop] = new_cost
                    priority = new_cost + heuristic_p(graph, connection, goal)
                    frontier.put((next_stop, line), priority)
                    came_from[next_stop] = (current_stop, connection)
            if current_stop in came_from:
                start_time = (came_from[current_stop][1]).arrival_time 
        
                    
    current_stop = goal
    if current_stop not in came_from:
        return []
    while current_stop != start:
        path.append(came_from[current_stop][1])
        current_stop = came_from[current_stop][0]
    
    path.reverse()
    
    journey_end_time = path[-1].arrival_time
    total_journey_time = journey_end_time - journey_start_time
    
    return path, total_journey_time

Obviously, now let's check some sample data to see how (if) it works.

In [44]:
start_stop = "GALERIA DOMINIKAŃSKA"
goal = "Kominiarska"
#criterion = "p"
#if criterion != 'p' and criterion != 't':
#    raise argparse.ArgumentTypeError("Optimization criterion should be either 't' or 'p'.")

start_time = "24:00"
start_time = parse_time(start_time)

minimized_criterion_value = "Travel Time" if criterion == 't' else "Number of Transfers"

start = time.time()
a_star_p_path, a_star_p_journey = a_star_transfer(graph, start_stop, goal, start_time)
end = time.time()
req_time = end - start


print_path(a_star_p_path)

print()
print(f"Starting time: {start_time}")
print(f"Total journey time: {a_star_p_journey}")
print(f"Minimized Criterion: {minimized_criterion_value}", file=sys.stderr)
print(f"Time Required: {req_time}", file=sys.stderr)

GALERIA DOMINIKAŃSKA -> Hala Targowa, Line: 244, Departure: 00:23:00, Arrival: 00:24:00
Hala Targowa -> Dubois, Line: 244, Departure: 00:24:00, Arrival: 00:26:00
Dubois -> pl. Bema, Line: 244, Departure: 00:26:00, Arrival: 00:28:00
pl. Bema -> Na Szańcach, Line: 244, Departure: 00:28:00, Arrival: 00:28:00
Na Szańcach -> Jedności Narodowej, Line: 244, Departure: 00:28:00, Arrival: 00:29:00
Jedności Narodowej -> Nowowiejska, Line: 244, Departure: 00:29:00, Arrival: 00:31:00
Nowowiejska -> Daszyńskiego, Line: 244, Departure: 00:31:00, Arrival: 00:32:00
Daszyńskiego -> Słonimskiego, Line: 244, Departure: 00:32:00, Arrival: 00:33:00
Słonimskiego -> Zakładowa, Line: 244, Departure: 00:33:00, Arrival: 00:34:00
Zakładowa -> Broniewskiego, Line: 244, Departure: 00:34:00, Arrival: 00:36:00
Broniewskiego -> Bałtycka, Line: 244, Departure: 00:36:00, Arrival: 00:38:00
Bałtycka -> Bezpieczna, Line: 244, Departure: 00:38:00, Arrival: 00:39:00
Bezpieczna -> Paprotna, Line: 244, Departure: 00:39:00, Ar

Minimized Criterion: Number of Transfers
Time Required: 0.1385188102722168


In [46]:
start_stop = "Psie Pole"
goal = "Kamieńskiego"
criterion = "p"
if criterion != 'p' and criterion != 't':
    raise argparse.ArgumentTypeError("Optimization criterion should be either 't' or 'p'.")

start_time = "22:22:22"
start_time = parse_time(start_time)

minimized_criterion_value = "Travel Time" if criterion == 't' else "Number of Transfers"

start = time.time()
a_star_p_path, a_star_p_journey = a_star_transfer(graph, start_stop, goal, start_time)
end = time.time()
req_time = end - start


print_path(a_star_p_path)

print()
print(f"Starting time: {start_time}")
print(f"Total journey time: {a_star_p_journey}")
print(f"Minimized Criterion: {minimized_criterion_value}", file=sys.stderr)
print(f"Time Required: {req_time}", file=sys.stderr)

Psie Pole -> Zielna, Line: 128 , Departure: 23:12:00, Arrival: 23:12:00
Zielna -> C.H. Korona, Line: 128 , Departure: 23:12:00, Arrival: 23:14:00
C.H. Korona -> Brücknera, Line: 128 , Departure: 23:15:00, Arrival: 23:16:00
Brücknera -> Grudziądzka, Line: 128 , Departure: 23:16:00, Arrival: 23:17:00
Grudziądzka -> Kromera (Czajkowskiego), Line: 128 , Departure: 23:17:00, Arrival: 23:18:00
Kromera (Czajkowskiego) -> KROMERA, Line: 128 , Departure: 23:18:00, Arrival: 23:19:00
KROMERA -> Berenta, Line: 128 , Departure: 23:19:00, Arrival: 23:20:00
Berenta -> Kasprowicza, Line: 128 , Departure: 23:20:00, Arrival: 23:21:00
Kasprowicza -> Syrokomli, Line: 128 , Departure: 23:21:00, Arrival: 23:22:00
Syrokomli -> Pola, Line: 128 , Departure: 23:22:00, Arrival: 23:23:00
Pola -> Broniewskiego, Line: 128 , Departure: 23:23:00, Arrival: 23:24:00
Broniewskiego -> Kamieńskiego, Line: 128 , Departure: 23:24:00, Arrival: 23:25:00

Starting time: 2023-03-02 22:22:22
Total journey time: 1:02:38


Minimized Criterion: Number of Transfers
Time Required: 0.10631179809570312


In [48]:
start_stop = "Reja"
goal = "Krakowska"
criterion = "p"
if criterion != 'p' and criterion != 't':
    raise argparse.ArgumentTypeError("Optimization criterion should be either 't' or 'p'.")

start_time = "26:30"
start_time = parse_time(start_time)

minimized_criterion_value = "Travel Time" if criterion == 't' else "Number of Transfers"

start = time.time()
a_star_p_path, a_star_p_journey = a_star_transfer(graph, start_stop, goal, start_time)
end = time.time()
req_time = end - start


print_path(a_star_p_path)

print()
print(f"Starting time: {start_time}")
print(f"Total journey time: {a_star_p_journey}")
print(f"Minimized Criterion: {minimized_criterion_value}", file=sys.stderr)
print(f"Time Required: {req_time}", file=sys.stderr)

Reja -> Katedra, Line: 253, Departure: 02:41:00, Arrival: 02:42:00
Katedra -> Urząd Wojewódzki (Muzeum Narodowe), Line: 253, Departure: 02:42:00, Arrival: 02:43:00
Urząd Wojewódzki (Muzeum Narodowe) -> pl. Wróblewskiego, Line: 240, Departure: 02:56:00, Arrival: 02:58:00
pl. Wróblewskiego -> pl. Zgody (Muzeum Etnograficzne), Line: 243, Departure: 03:10:00, Arrival: 03:11:00
pl. Zgody (Muzeum Etnograficzne) -> Na Niskich Łąkach, Line: 243, Departure: 03:11:00, Arrival: 03:13:00
Na Niskich Łąkach -> Krakowska, Line: 243, Departure: 03:13:00, Arrival: 03:14:00

Starting time: 2023-03-03 02:30:00
Total journey time: 0:44:00


Minimized Criterion: Number of Transfers
Time Required: 0.5571398735046387


Here we can it with our own input:

In [ ]:
start_stop = input("Please provide the starting stop: ")
goal = input("Please provide the ending stop: ")
#criterion = input(f"Optimization criterion:\n\tt (travel time)\n\tp (number of transfers)")
#if criterion != 'p' and criterion != 't':
#    raise argparse.ArgumentTypeError("Optimization criterion should be either 't' or 'p'.")

start_time = input("Please provide time in form HH:MM or HH:MM:SS: ")
start_time = parse_time(start_time)


minimized_criterion_value = "Travel Time" if criterion == 't' else "Number of Transfers"

In [ ]:
a_star_p_path, a_star_p_journey = a_star_transfer(graph, start_stop, goal, start_time)

In [ ]:
print_path(a_star_p_path)

print()
print(f"Starting time: {start_time}")
print(f"Total journey time: {a_star_p_journey}")

### General testing the algorithms
So now, we can provide our input with criterion of optimization (`t` for time optimization, `p` for number of transfers optimization). If we choose `t`, then the algorithm will choose the one with shorter journey time, and if both are the same, the one with shorter time of processing, and if both are the same, the A* search algorithm is chosen. 

In [ ]:
start_stop = input("Please provide the starting stop: ")
goal = input("Please provide the ending stop: ")
criterion = input(f"Optimization criterion:\n\tt (travel time)\n\tp (number of transfers)")
if criterion != 'p' and criterion != 't':
    raise argparse.ArgumentTypeError("Optimization criterion should be either 't' or 'p'.")

start_time = input("Please provide time in form HH:MM or HH:MM:SS: ")
start_time = parse_time(start_time)


minimized_criterion_value = "Travel Time" if criterion == 't' else "Number of Transfers"

In [ ]:
req_time = 0
journey_time = 0
if criterion == 't':
        start_d = time.time()
        dijkstra_path, dijkstra_journey = dijkstra(graph, start_stop, goal, start_time)
        end_d = time.time()
        dijkstra_time = end_d - start_d
        
        start_a = time.time()
        a_star_time_path, a_star_time_journey = a_star_time(graph, start_stop, goal, start_time)
        end_a = time.time()
        a_star_time_time = end_a - start_a
        
        if dijkstra_journey < a_star_time_journey:
            algorithm = "Dijkstra"
            req_time = dijkstra_time
            journey_time = dijkstra_journey
            print_path(dijkstra_path)
        elif dijkstra_journey == a_star_time_journey:
            if dijkstra_time < a_star_time_time:
                algorithm = "Dijkstra"
                req_time = dijkstra_time
                journey_time = dijkstra_journey
                print_path(dijkstra_path)
            else:
                algorithm = "A*"
                req_time = a_star_time_time
                journey_time = a_star_time_journey
                print_path(a_star_time_path)
        else:
            algorithm = "A*"
            req_time = a_star_time_time
            journey_time = a_star_time_journey
            print_path(a_star_time_path)
else:
        algorithm = "A*"
        start = time.time()
        a_star_transfer_path, a_star_transfer_journey = a_star_transfer(graph, start_stop, goal, start_time)
        end = time.time()
        req_time = end - start
        journey_time = a_star_transfer_journey
        print_path(a_star_transfer_path)

# Printing minimized criterion and time required for calculation
print()
print(f"Starting time: {start_time}")
print(f"Total journey time: {journey_time}")
print(f"Used algorithm: {algorithm}")
print(f"Minimized Criterion: {minimized_criterion_value}", file=sys.stderr)
print(f"Time Required: {req_time}", file=sys.stderr)

## Task 2 - Tabu Search
The application should accept input in form of 4 variables:
- the starting stop A (marked as `start_stop`)
- list of stops to visit separated by semicolon L (marked as `stops_to_visit`)
- optimization criterion (value `t` denotes travel time minimization, value `p` means minimization of the number of transfers)
- starting time.

### Tabu Search without bounding of the size T

So we are going to use the following objective function:

In [6]:
def objective_function(path, start_time, criterion):
    if criterion == 't':
        total_time = path[-1].arrival_time - start_time
        return total_time.total_seconds()
    elif criterion == 'p':  
        transfers = len(set(connection.line for connection in path)) - 1
        return transfers

The algorithm:

In [7]:
def tabu_search(graph, start_stop, stops_to_visit, criterion, start_time):
    current_stop = start_stop
    current_time = start_time
    visited_stops = [current_stop]
    current_path = []
    tabu_list = []

    while stops_to_visit:
        neighbors = graph.get_connections(current_stop)
        best_neighbor = None
        best_cost = float('inf')

        for neighbor in neighbors:
            if neighbor.end_stop in visited_stops or (current_stop, neighbor.end_stop) in tabu_list:
                continue

            if neighbor.end_stop in stops_to_visit:
                if current_time <= neighbor.departure_time:
                    new_path = current_path + [neighbor]
                    cost = objective_function(new_path, current_time, criterion)
                    if cost < best_cost:
                        best_cost = cost
                        best_neighbor = neighbor

        if best_neighbor:
            visited_stops.append(best_neighbor.end_stop)
            current_path.append(best_neighbor)
            current_stop = best_neighbor.end_stop
            current_time = best_neighbor.arrival_time
            stops_to_visit.remove(current_stop)
            tabu_list.append((current_stop, current_path[-1].end_stop))
            if len(tabu_list) > len(graph.graph):
                tabu_list.pop(0)

    total_journey_time = current_time - start_time
    return current_path, total_journey_time

Now let's test it with some sample data.

In [8]:
start_stop = "Kasprowicza"
stops_to_visit = "Syrokomli;Pola;Broniewskiego;Bałtycka"

try:
    stops_to_visit = stops_to_visit.split(';')
except AttributeError:
    print("Error: Stops to visit must be provided as a semicolon-separated list.", file=sys.stderr)
    sys.exit(1)
except Exception as e:
    print(f"An unexpected error occurred: {str(e)}", file=sys.stderr)
    sys.exit(1)
    
criterion = "t"
if criterion != 'p' and criterion != 't':
    raise argparse.ArgumentTypeError("Optimization criterion should be either 't' or 'p'.")

start_time = "12:07:00"
start_time = parse_time(start_time)

minimized_criterion_value = "Travel Time" if criterion == 't' else "Number of Transfers"

start = time.time()
tabu_path, tabu_time = tabu_search(graph, start_stop, stops_to_visit, criterion, start_time)
end = time.time()

req_time = end - start


print_path(tabu_path)

print()
print(f"Starting time: {start_time}")
print(f"Total journey time: {tabu_time}")
print(f"Minimized Criterion: {minimized_criterion_value}", file=sys.stderr)
print(f"Time Required: {req_time}", file=sys.stderr)

Kasprowicza -> Syrokomli, Line: 105, Departure: 12:07:00, Arrival: 12:08:00
Syrokomli -> Pola, Line: 105, Departure: 12:08:00, Arrival: 12:09:00
Pola -> Broniewskiego, Line: 105, Departure: 12:09:00, Arrival: 12:11:00
Broniewskiego -> Bałtycka, Line: 105, Departure: 12:11:00, Arrival: 12:13:00

Starting time: 2023-03-02 12:07:00
Total journey time: 0:06:00


Minimized Criterion: Travel Time
Time Required: 0.009004354476928711


In [38]:
start_stop = "Bałtycka"
stops_to_visit = "Pola;Kasprowicza;Broniewskiego;Syrokomli"

try:
    stops_to_visit = stops_to_visit.split(';')
except AttributeError:
    print("Error: Stops to visit must be provided as a semicolon-separated list.", file=sys.stderr)
    sys.exit(1)
except Exception as e:
    print(f"An unexpected error occurred: {str(e)}", file=sys.stderr)
    sys.exit(1)
    
criterion = "p"
if criterion != 'p' and criterion != 't':
    raise argparse.ArgumentTypeError("Optimization criterion should be either 't' or 'p'.")

start_time = "20:30"
start_time = parse_time(start_time)

minimized_criterion_value = "Travel Time" if criterion == 't' else "Number of Transfers"

start = time.time()
tabu_path, tabu_time = tabu_search(graph, start_stop, stops_to_visit, criterion, start_time)
end = time.time()

req_time = end - start


print_path(tabu_path)

print()
print(f"Starting time: {start_time}")
print(f"Total journey time: {tabu_time}")
print(f"Minimized Criterion: {minimized_criterion_value}", file=sys.stderr)
print(f"Time Required: {req_time}", file=sys.stderr)

Bałtycka -> Broniewskiego, Line: A, Departure: 20:57:00, Arrival: 20:59:00
Broniewskiego -> Pola, Line: A, Departure: 20:59:00, Arrival: 21:00:00
Pola -> Syrokomli, Line: A, Departure: 21:00:00, Arrival: 21:01:00
Syrokomli -> Kasprowicza, Line: A, Departure: 21:01:00, Arrival: 21:02:00

Starting time: 2023-03-02 20:30:00
Total journey time: 0:32:00


Minimized Criterion: Number of Transfers
Time Required: 0.006506204605102539


### Tabu Search with bouding of the size T

Now we can introduce a tiny modification in our code in order to minimize the cost function—we will use bounding based on the length of `stops_to_visit`.

In [9]:
def tabu_search_bounding(graph, start_stop, stops_to_visit, criterion, start_time):
    current_stop = start_stop
    current_time = start_time
    visited_stops = [current_stop]
    current_path = []
    tabu_list_size = len(stops_to_visit) # bounding
    tabu_list = []

    while stops_to_visit:
        neighbors = graph.get_connections(current_stop)
        best_neighbor = None
        best_cost = float('inf')

        for neighbor in neighbors:
            if neighbor.end_stop in visited_stops or (current_stop, neighbor.end_stop) in tabu_list:
                continue

            if neighbor.end_stop in stops_to_visit:
                if current_time <= neighbor.departure_time:
                    new_path = current_path + [neighbor]
                    cost = objective_function(new_path, current_time, criterion)
                    if cost < best_cost:
                        best_cost = cost
                        best_neighbor = neighbor

        if best_neighbor:
            visited_stops.append(best_neighbor.end_stop)
            current_path.append(best_neighbor)
            current_stop = best_neighbor.end_stop
            current_time = best_neighbor.arrival_time
            stops_to_visit.remove(current_stop)
            tabu_list.append((current_stop, current_path[-1].end_stop))
            if len(tabu_list) > tabu_list_size:
                tabu_list.pop(0)

    total_journey_time = current_time - start_time
    
    return current_path, total_journey_time

Let's test it:

In [11]:
start_stop = "Pereca"
stops_to_visit = "Pułaskiego;Kolejowa;Grabiszyńska;pl. Legionów;Kościuszki;Arkady (Capitol);DWORZEC GŁÓWNY"

try:
    stops_to_visit = stops_to_visit.split(';')
except AttributeError:
    print("Error: Stops to visit must be provided as a semicolon-separated list.", file=sys.stderr)
    sys.exit(1)
except Exception as e:
    print(f"An unexpected error occurred: {str(e)}", file=sys.stderr)
    sys.exit(1)
    
criterion = "t"
if criterion != 'p' and criterion != 't':
    raise argparse.ArgumentTypeError("Optimization criterion should be either 't' or 'p'.")

start_time = "12:07:00"
start_time = parse_time(start_time)

minimized_criterion_value = "Travel Time" if criterion == 't' else "Number of Transfers"

start = time.time()
tabu_path, tabu_time = tabu_search_bounding(graph, start_stop, stops_to_visit, criterion, start_time)
end = time.time()

req_time = end - start


print_path(tabu_path)

print()
print(f"Starting time: {start_time}")
print(f"Total journey time: {tabu_time}")
print(f"Minimized Criterion: {minimized_criterion_value}", file=sys.stderr)
print(f"Time Required: {req_time}", file=sys.stderr)

Pereca -> Grabiszyńska, Line: 11, Departure: 12:07:00, Arrival: 12:08:00
Grabiszyńska -> Kolejowa, Line: 11, Departure: 12:08:00, Arrival: 12:09:00
Kolejowa -> pl. Legionów, Line: 11, Departure: 12:09:00, Arrival: 12:11:00
pl. Legionów -> Arkady (Capitol), Line: 21, Departure: 12:11:00, Arrival: 12:13:00
Arkady (Capitol) -> DWORZEC GŁÓWNY, Line: 23, Departure: 12:16:00, Arrival: 12:19:00
DWORZEC GŁÓWNY -> Pułaskiego, Line: 22, Departure: 12:22:00, Arrival: 12:24:00
Pułaskiego -> Kościuszki, Line: 4, Departure: 12:28:00, Arrival: 12:30:00

Starting time: 2023-03-02 12:07:00
Total journey time: 0:23:00


Minimized Criterion: Travel Time
Time Required: 0.02151799201965332


In [15]:
start_stop = "Bałtycka"
stops_to_visit = "Pola;Kasprowicza;Broniewskiego;Syrokomli"

try:
    stops_to_visit = stops_to_visit.split(';')
except AttributeError:
    print("Error: Stops to visit must be provided as a semicolon-separated list.", file=sys.stderr)
    sys.exit(1)
except Exception as e:
    print(f"An unexpected error occurred: {str(e)}", file=sys.stderr)
    sys.exit(1)
    
criterion = "p"
if criterion != 'p' and criterion != 't':
    raise argparse.ArgumentTypeError("Optimization criterion should be either 't' or 'p'.")

start_time = "20:30"
start_time = parse_time(start_time)

minimized_criterion_value = "Travel Time" if criterion == 't' else "Number of Transfers"

start = time.time()
tabu_path, tabu_time = tabu_search_bounding(graph, start_stop, stops_to_visit, criterion, start_time)
end = time.time()

req_time = end - start


print_path(tabu_path)

print()
print(f"Starting time: {start_time}")
print(f"Total journey time: {tabu_time}")
print(f"Minimized Criterion: {minimized_criterion_value}", file=sys.stderr)
print(f"Time Required: {req_time}", file=sys.stderr)

Bałtycka -> Broniewskiego, Line: A, Departure: 20:57:00, Arrival: 20:59:00
Broniewskiego -> Pola, Line: A, Departure: 20:59:00, Arrival: 21:00:00
Pola -> Syrokomli, Line: A, Departure: 21:00:00, Arrival: 21:01:00
Syrokomli -> Kasprowicza, Line: A, Departure: 21:01:00, Arrival: 21:02:00

Starting time: 2023-03-02 20:30:00
Total journey time: 0:32:00


Minimized Criterion: Number of Transfers
Time Required: 0.008001327514648438


### Aspiration criterion
Now another modification—let's add Aspiration criterion (in order to minimize the cost function).

In [16]:
def calculate_aspiration_criterion(cost, history_count, epsilon, k):
    return cost + epsilon * (k - history_count)

In [18]:
def tabu_search_aspiration(graph, start_stop, stops_to_visit, criterion, start_time, epsilon, k):
    current_stop = start_stop
    current_time = start_time
    visited_stops = [current_stop]
    current_path = []
    tabu_list_size = len(stops_to_visit)
    tabu_list = []
    aspiration_history = [0] * k
    best_cost = float('inf')
    best_solution = None

    while stops_to_visit:
        neighbors = graph.get_connections(current_stop)
        best_neighbor = None
        best_neighbor_cost = float('inf')

        for neighbor in neighbors:
            if neighbor.end_stop in visited_stops or (current_stop, neighbor.end_stop) in tabu_list:
                continue

            if neighbor.end_stop in stops_to_visit:
                if current_time <= neighbor.departure_time:
                    new_path = current_path + [neighbor]
                    cost = objective_function(new_path, current_time, criterion)
                    
                    asp_index = stops_to_visit.index(neighbor.end_stop)
                    asp_criterion = calculate_aspiration_criterion(cost, aspiration_history[asp_index], epsilon, k)
                    
                    if cost < best_neighbor_cost or cost < asp_criterion:
                        best_neighbor_cost = cost
                        best_neighbor = neighbor

        if best_neighbor:
            if current_stop in stops_to_visit:
                asp_index = stops_to_visit.index(current_stop)
                aspiration_history[asp_index] += 1

            visited_stops.append(best_neighbor.end_stop)
            current_path.append(best_neighbor)
            current_stop = best_neighbor.end_stop
            current_time = best_neighbor.arrival_time
            stops_to_visit.remove(current_stop)
            tabu_list.append((current_stop, current_path[-1].end_stop))
            if len(tabu_list) > tabu_list_size:
                tabu_list.pop(0)
            
            # checking aspiration criterion
            if best_neighbor_cost < best_cost or best_neighbor_cost < asp_criterion:
                best_solution = current_path[:]
                best_cost = best_neighbor_cost

    total_journey_time = current_time - start_time
    
    return best_solution, total_journey_time

Let's test it with some sample data.

In [22]:
start_stop = "Pereca"
stops_to_visit = "Grabiszyńska;Kolejowa"

try:
    stops_to_visit = stops_to_visit.split(';')
except AttributeError:
    print("Error: Stops to visit must be provided as a semicolon-separated list.", file=sys.stderr)
    sys.exit(1)
except Exception as e:
    print(f"An unexpected error occurred: {str(e)}", file=sys.stderr)
    sys.exit(1)
    
criterion = "t"
if criterion != 'p' and criterion != 't':
    raise argparse.ArgumentTypeError("Optimization criterion should be either 't' or 'p'.")

k = len(stops_to_visit)

epsilon = 0.5
if epsilon >= 1 or epsilon <= 0:
    raise argparse.ArgumentTypeError("Epsilon must be between 0 and 1.")

start_time = "23:07:00"
start_time = parse_time(start_time)

minimized_criterion_value = "Travel Time" if criterion == 't' else "Number of Transfers"

start = time.time()
tabu_path, tabu_time = tabu_search_aspiration(graph, start_stop, stops_to_visit, criterion, start_time, epsilon, k)
end = time.time()

req_time = end - start


print_path(tabu_path)

print()
print(f"Starting time: {start_time}")
print(f"Total journey time: {tabu_time}")
print(f"Minimized Criterion: {minimized_criterion_value}", file=sys.stderr)
print(f"Time Required: {req_time}", file=sys.stderr)

Pereca -> Grabiszyńska, Line: 251, Departure: 23:43:00, Arrival: 23:44:00
Grabiszyńska -> Kolejowa, Line: 11, Departure: 23:52:00, Arrival: 23:53:00

Starting time: 2023-03-02 23:07:00
Total journey time: 0:46:00


Minimized Criterion: Travel Time
Time Required: 0.003004789352416992


In [74]:
start_stop = "Kasprowicza"
stops_to_visit = "Syrokomli;Broniewskiego;Bałtycka;Pola"

try:
    stops_to_visit = stops_to_visit.split(';')
except AttributeError:
    print("Error: Stops to visit must be provided as a semicolon-separated list.", file=sys.stderr)
    sys.exit(1)
except Exception as e:
    print(f"An unexpected error occurred: {str(e)}", file=sys.stderr)
    sys.exit(1)
    
criterion = "p"
if criterion != 'p' and criterion != 't':
    raise argparse.ArgumentTypeError("Optimization criterion should be either 't' or 'p'.")

k = len(stops_to_visit)

epsilon = 0.5
if epsilon >= 1 or epsilon <= 0:
    raise argparse.ArgumentTypeError("Epsilon must be between 0 and 1.")

start_time = "19:07:00"
start_time = parse_time(start_time)

minimized_criterion_value = "Travel Time" if criterion == 't' else "Number of Transfers"

start = time.time()
tabu_path, tabu_time = tabu_search_aspiration(graph, start_stop, stops_to_visit, criterion, start_time, epsilon, k)
end = time.time()

req_time = end - start


print_path(tabu_path)

print()
print(f"Starting time: {start_time}")
print(f"Total journey time: {tabu_time}")
print(f"Minimized Criterion: {minimized_criterion_value}", file=sys.stderr)
print(f"Time Required: {req_time}", file=sys.stderr)

Kasprowicza -> Syrokomli, Line: 315, Departure: 19:08:00, Arrival: 19:09:00
Syrokomli -> Pola, Line: 315, Departure: 19:09:00, Arrival: 19:10:00
Pola -> Broniewskiego, Line: 315, Departure: 19:10:00, Arrival: 19:12:00
Broniewskiego -> Bałtycka, Line: 315, Departure: 19:12:00, Arrival: 19:14:00

Starting time: 2023-03-02 19:07:00
Total journey time: 0:07:00


Minimized Criterion: Number of Transfers
Time Required: 0.009521007537841797


### Sampling strategy
Now another modification—we will add a neighbor sampling strategy.

In [23]:
def tabu_search_with_neighbor_sampling(graph, start_stop, stops_to_visit, criterion, start_time, sampling_ratio=0.5):
    current_stop = start_stop
    current_time = start_time
    visited_stops = [current_stop]
    current_path = []
    tabu_list = []

    while stops_to_visit:
        neighbors = graph.get_connections(current_stop)
        sampled_neighbors = random.sample(neighbors, int(len(neighbors) * sampling_ratio))
        best_neighbor = None
        best_cost = float('inf')

        for neighbor in sampled_neighbors:
            if neighbor.end_stop in visited_stops or (current_stop, neighbor.end_stop) in tabu_list:
                continue

            if neighbor.end_stop in stops_to_visit:
                if current_time <= neighbor.departure_time:
                    new_path = current_path + [neighbor]
                    cost = objective_function(new_path, current_time, criterion)
                    if cost < best_cost:
                        best_cost = cost
                        best_neighbor = neighbor

        if best_neighbor:
            visited_stops.append(best_neighbor.end_stop)
            current_path.append(best_neighbor)
            current_stop = best_neighbor.end_stop
            current_time = best_neighbor.arrival_time
            stops_to_visit.remove(current_stop)
            tabu_list.append((current_stop, current_path[-1].end_stop))
            if len(tabu_list) > len(graph.graph):
                tabu_list.pop(0)

    total_journey_time = current_time - start_time
    return current_path, total_journey_time

And some sample testing data:

In [24]:
start_stop = "Pereca"
stops_to_visit = "Pułaskiego;Kolejowa;Grabiszyńska;pl. Legionów;Kościuszki;Arkady (Capitol);DWORZEC GŁÓWNY"

try:
    stops_to_visit = stops_to_visit.split(';')
except AttributeError:
    print("Error: Stops to visit must be provided as a semicolon-separated list.", file=sys.stderr)
    sys.exit(1)
except Exception as e:
    print(f"An unexpected error occurred: {str(e)}", file=sys.stderr)
    sys.exit(1)
    
criterion = "t"
if criterion != 'p' and criterion != 't':
    raise argparse.ArgumentTypeError("Optimization criterion should be either 't' or 'p'.")

start_time = "12:07:00"
start_time = parse_time(start_time)

minimized_criterion_value = "Travel Time" if criterion == 't' else "Number of Transfers"

start = time.time()
tabu_path, tabu_time = tabu_search_with_neighbor_sampling(graph, start_stop, stops_to_visit, criterion, start_time)
end = time.time()

req_time = end - start


print_path(tabu_path)

print()
print(f"Starting time: {start_time}")
print(f"Total journey time: {tabu_time}")
print(f"Minimized Criterion: {minimized_criterion_value}", file=sys.stderr)
print(f"Time Required: {req_time}", file=sys.stderr)

Pereca -> Grabiszyńska, Line: 11, Departure: 12:07:00, Arrival: 12:08:00
Grabiszyńska -> Kolejowa, Line: 11, Departure: 12:08:00, Arrival: 12:09:00
Kolejowa -> pl. Legionów, Line: 11, Departure: 12:09:00, Arrival: 12:11:00
pl. Legionów -> Arkady (Capitol), Line: 15, Departure: 12:13:00, Arrival: 12:15:00
Arkady (Capitol) -> DWORZEC GŁÓWNY, Line: 23, Departure: 12:16:00, Arrival: 12:19:00
DWORZEC GŁÓWNY -> Pułaskiego, Line: 22, Departure: 12:22:00, Arrival: 12:24:00
Pułaskiego -> Kościuszki, Line: 4, Departure: 12:28:00, Arrival: 12:30:00

Starting time: 2023-03-02 12:07:00
Total journey time: 0:23:00


Minimized Criterion: Travel Time
Time Required: 0.018000364303588867


In [30]:
start_stop = "Kościuszki"
stops_to_visit = "Komuny Paryskiej;pl. Wróblewskiego;Urząd Wojewódzki (Impart)"

try:
    stops_to_visit = stops_to_visit.split(';')
except AttributeError:
    print("Error: Stops to visit must be provided as a semicolon-separated list.", file=sys.stderr)
    sys.exit(1)
except Exception as e:
    print(f"An unexpected error occurred: {str(e)}", file=sys.stderr)
    sys.exit(1)
    
criterion = "p"
if criterion != 'p' and criterion != 't':
    raise argparse.ArgumentTypeError("Optimization criterion should be either 't' or 'p'.")

start_time = "12:07:00"
start_time = parse_time(start_time)

minimized_criterion_value = "Travel Time" if criterion == 't' else "Number of Transfers"

start = time.time()
tabu_path, tabu_time = tabu_search_with_neighbor_sampling(graph, start_stop, stops_to_visit, criterion, start_time)
end = time.time()

req_time = end - start


print_path(tabu_path)

print()
print(f"Starting time: {start_time}")
print(f"Total journey time: {tabu_time}")
print(f"Minimized Criterion: {minimized_criterion_value}", file=sys.stderr)
print(f"Time Required: {req_time}", file=sys.stderr)

Kościuszki -> Komuny Paryskiej, Line: 16, Departure: 14:42:00, Arrival: 14:43:00
Komuny Paryskiej -> pl. Wróblewskiego, Line: 16, Departure: 17:12:00, Arrival: 17:14:00
pl. Wróblewskiego -> Urząd Wojewódzki (Impart), Line: 16, Departure: 18:08:00, Arrival: 18:10:00

Starting time: 2023-03-02 12:07:00
Total journey time: 6:03:00


Minimized Criterion: Number of Transfers
Time Required: 0.0029997825622558594
